In [9]:
# Module Importations
import sklearn
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as keras

# Module Versioning
print('sklearn: ' + sklearn.__version__)
print('keras version: ', keras.__version__)
print('tensorflow version: ', tf.__version__) 

sklearn: 0.22.2.post1
keras version:  2.4.0
tensorflow version:  2.4.1


In [10]:
# Custom Modules
from Source.Data import ntfp_dataset_import as data_load
from Source.Data import ntfp_split_data as split_data
from Source.Model import ntfp_keras_helpers as keras_helpers

In [11]:
# Constants
TRAIN_MODELS = True

In [12]:
# Load Dataset (Use set from previous ML investigation)
filename = 'rul_dataset.pkl'
rul_df = data_load.load_pickled_data(filename)

print(rul_df)

Loading pickled dataframe started ...
Loading pickled dataframe complete ...
        Cycles   Sn_02    Sn_03    Sn_04   Sn_07    Sn_09  Sn_11   Sn_12  \
Engine                                                                     
1            1  641.82  1589.70  1400.60  554.36  9046.19  47.47  521.66   
1            2  642.15  1591.82  1403.14  553.75  9044.07  47.49  522.28   
1            3  642.35  1587.99  1404.20  554.26  9052.94  47.27  522.42   
1            4  642.35  1582.79  1401.87  554.45  9049.48  47.13  522.86   
1            5  642.37  1582.85  1406.22  554.00  9055.15  47.28  522.19   
...        ...     ...      ...      ...     ...      ...    ...     ...   
100        196  643.49  1597.98  1428.63  551.43  9065.52  48.07  519.49   
100        197  643.54  1604.50  1433.58  550.86  9065.11  48.04  519.68   
100        198  643.42  1602.46  1428.18  550.94  9065.90  48.09  520.01   
100        199  643.23  1605.26  1426.53  550.68  9073.72  48.39  519.67   
100        

In [13]:
# Split data into training and evaluation sets
training_set, evaluation_set = split_data.split_train_eval(rul_df, 0.2)

Original Data Items: 20631
Training Data Items: 16505
Evaluation Data Items: 4126


In [14]:
# Create RUL Target Dataset

# Training Set as Array
rul_training_data = training_set.drop('RUL', axis = 1).values
rul_training_label = training_set['RUL'].copy().values

print(rul_training_data.shape, rul_training_label.shape)

# Evaluation Set as Array
rul_eval_data = evaluation_set.drop('RUL', axis = 1).values
rul_eval_label = evaluation_set['RUL'].copy().values

print(rul_eval_data.shape, rul_eval_label.shape)

(16505, 13) (16505,)
(4126, 13) (4126,)


In [15]:
# Create train and test arrays
X_train, X_test, y_train, y_test = train_test_split(rul_training_data, rul_training_label, test_size = 0.2, random_state = 0)

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(13204, 13) (3301, 13)
(13204,) (3301,)


In [16]:
# Initial MLP (Target - RUL)

if TRAIN_MODELS == True:
    # Clear existing models
    keras.backend.clear_session()

    # Build model
    model = keras_helpers.build_multilayer_perceptron()

    # Name model
    model_type = "MLP_RUL"
    model_id = keras_helpers.name_model(model_type)
    filepath_full = keras_helpers.make_save_string(model_id)

    # Set save and earlystop callbacks
    earlystop_cb = keras.callbacks.EarlyStopping(patience = 5)
    checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath = filepath_full, save_best_only = True)

    # Train model
    model.fit(X_train, y_train, epochs = 20, validation_data =(X_test, y_test), callbacks =[checkpoint_cb, earlystop_cb])

Building Model ...
Hidden Layers: 2, Neurons: 6, LR: 0.001
Epoch 1/20
413/413 [==============================] - 1s 2ms/step - loss: 3347404.3065 - val_loss: 4798.5210
Epoch 2/20
413/413 [==============================] - 1s 1ms/step - loss: 4642.6370 - val_loss: 4494.7217
Epoch 3/20
413/413 [==============================] - 1s 1ms/step - loss: 4422.2877 - val_loss: 3636.6199
Epoch 4/20
413/413 [==============================] - 1s 1ms/step - loss: 2832.2579 - val_loss: 2269.0498
Epoch 5/20
413/413 [==============================] - 1s 1ms/step - loss: 2088.2265 - val_loss: 1984.4752
Epoch 6/20
413/413 [==============================] - 0s 1ms/step - loss: 2137.9431 - val_loss: 1992.4800
Epoch 7/20
413/413 [==============================] - 1s 1ms/step - loss: 2178.3602 - val_loss: 2067.3777
Epoch 8/20
413/413 [==============================] - 0s 1ms/step - loss: 2153.7649 - val_loss: 1984.1631
Epoch 9/20
413/413 [==============================] - 1s 2ms/step - loss: 2183.9838 - val_